# DSFB Fusion Diagnostics Figures

This notebook is plotting-only. It loads benchmark outputs generated by `dsfb-fusion-bench` and saves publication-ready figures.

## Reproduce benchmark outputs first

```bash
cargo run --release -p dsfb-fusion-bench -- --run-default
cargo run --release -p dsfb-fusion-bench -- --run-sweep
```

Expected files in a timestamped run directory:
- `output-dsfb-fusion-bench/<timestamp>/summary.csv`
- `output-dsfb-fusion-bench/<timestamp>/heatmap.csv`
- `output-dsfb-fusion-bench/<timestamp>/sim-dsfb-fusion-bench.csv` (or `trajectories.csv`)


In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

FIG_DIR = Path("figures")
FIG_DIR.mkdir(parents=True, exist_ok=True)

plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "font.size": 10,
    "axes.grid": True,
    "grid.alpha": 0.3,
    "axes.spines.top": False,
    "axes.spines.right": False,
})


In [ ]:
# Optional URLs. Leave empty strings to use local files or upload.
DATA_URLS = {
    "summary": "",
    "heatmap": "",
    "trajectories": "",
}

REQUIRED_CORE = ("summary.csv", "heatmap.csv")
TRAJ_FILES = ("sim-dsfb-fusion-bench.csv", "trajectories.csv")
BASE_DIR_CANDIDATES = [
    Path('output-dsfb-fusion-bench'),
    Path('crates/dsfb-fusion-bench/output-dsfb-fusion-bench'),
    Path('outputs'),
    Path('crates/dsfb-fusion-bench/outputs'),
]

def trajectory_quality(path: Path) -> int:
    for name in TRAJ_FILES:
        fp = path / name
        if not fp.exists():
            continue
        try:
            with fp.open('r', encoding='utf-8') as f:
                lines = sum(1 for _ in f)
            return 2 if lines > 1 else 1
        except Exception:
            return 1
    return 0

def is_complete_run_dir(path: Path) -> bool:
    core_ok = all((path / name).exists() for name in REQUIRED_CORE)
    return core_ok and trajectory_quality(path) > 0

def find_latest_run_dir():
    env_dir = os.environ.get('DSFB_FUSION_BENCH_RUN_DIR', '').strip()
    if env_dir:
        env_path = Path(env_dir)
        if is_complete_run_dir(env_path):
            return env_path

    candidates = []
    for base_idx, base in enumerate(BASE_DIR_CANDIDATES):
        priority = len(BASE_DIR_CANDIDATES) - base_idx
        if is_complete_run_dir(base):
            candidates.append((priority, trajectory_quality(base), base.name, str(base), base))
        if not base.exists() or not base.is_dir():
            continue
        for d in base.iterdir():
            if d.is_dir() and is_complete_run_dir(d):
                candidates.append((priority, trajectory_quality(d), d.name, str(d), d))

    if not candidates:
        return None

    candidates = sorted(candidates)
    return candidates[-1][4]

RUN_DIR = find_latest_run_dir()
if RUN_DIR is not None:
    print(f'Using run directory: {RUN_DIR}')

DEFAULT_PATHS = {
    "summary": [],
    "heatmap": [],
    "trajectories": [],
}

if RUN_DIR is not None:
    DEFAULT_PATHS["summary"].append(str(RUN_DIR / "summary.csv"))
    DEFAULT_PATHS["heatmap"].append(str(RUN_DIR / "heatmap.csv"))
    DEFAULT_PATHS["trajectories"].append(str(RUN_DIR / "sim-dsfb-fusion-bench.csv"))
    DEFAULT_PATHS["trajectories"].append(str(RUN_DIR / "trajectories.csv"))

DEFAULT_PATHS["summary"] += ["summary.csv", "outputs/summary.csv"]
DEFAULT_PATHS["heatmap"] += ["heatmap.csv", "outputs/heatmap.csv"]
DEFAULT_PATHS["trajectories"] += ["sim-dsfb-fusion-bench.csv", "trajectories.csv", "outputs/sim-dsfb-fusion-bench.csv", "outputs/trajectories.csv"]

def load_csv(label: str) -> pd.DataFrame:
    url = DATA_URLS.get(label, '').strip()
    if url:
        print(f"Loading {label} from URL: {url}")
        return pd.read_csv(url)

    for p in DEFAULT_PATHS[label]:
        if Path(p).exists():
            print(f"Loading {label} from local path: {p}")
            return pd.read_csv(p)

    try:
        from google.colab import files
        print(f"Upload {label}.csv now...")
        uploaded = files.upload()
        if not uploaded:
            raise FileNotFoundError(f"No file uploaded for {label}")
        name = next(iter(uploaded.keys()))
        print(f"Loaded {label} from uploaded file: {name}")
        return pd.read_csv(name)
    except Exception as err:
        raise FileNotFoundError(
            f"Could not load {label}. Provide URL, local file, or upload in Colab."
        ) from err

summary_df = load_csv("summary")
heatmap_df = load_csv("heatmap")
traj_df = load_csv("trajectories")

print("summary shape:", summary_df.shape)
print("heatmap shape:", heatmap_df.shape)
print("trajectories shape:", traj_df.shape)


In [ ]:
# Numeric normalization for key metrics.
for col in ["peak_err", "rms_err", "baseline_wls_us", "overhead_us", "total_us", "false_downweight_rate"]:
    if col in summary_df.columns:
        summary_df[col] = pd.to_numeric(summary_df[col], errors="coerce")

for col in ["alpha", "beta", "peak_err", "rms_err", "false_downweight_rate"]:
    if col in heatmap_df.columns:
        heatmap_df[col] = pd.to_numeric(heatmap_df[col], errors="coerce")

for col in ["t", "err_norm"]:
    if col in traj_df.columns:
        traj_df[col] = pd.to_numeric(traj_df[col], errors="coerce")

print("\nSummary metrics by method (mean over available rows):")
metric_cols = [c for c in ["peak_err", "rms_err", "false_downweight_rate", "total_us"] if c in summary_df.columns]
print(summary_df.groupby("method", as_index=False)[metric_cols].mean().sort_values("method"))


In [ ]:
# Figure 1: Error trajectories over time for key methods.
key_methods = ["equal", "cov_inflate", "irls_huber", "nis_soft", "nis_hard", "dsfb"]
present = [m for m in key_methods if m in set(traj_df["method"].dropna().unique())]

plot_df = traj_df[traj_df["method"].isin(present)].copy()
plot_df = plot_df.groupby(["method", "t"], as_index=False)["err_norm"].mean()

fig, ax = plt.subplots(figsize=(8.0, 4.2))
for m in present:
    d = plot_df[plot_df["method"] == m]
    ax.plot(d["t"], d["err_norm"], label=m, linewidth=1.6)

ax.set_title("Error Norm Trajectories")
ax.set_xlabel("t")
ax.set_ylabel("||x_hat - x_true||")
ax.legend(ncol=2)
fig.tight_layout()
fig.savefig(FIG_DIR / "figure1_error_trajectories.png")
fig.savefig(FIG_DIR / "figure1_error_trajectories.pdf")
plt.show()


In [ ]:
# Figure 2: Trust weights over time for DSFB and soft NIS (if available).
weight_cols = [c for c in traj_df.columns if c.startswith("w_")]
methods_for_weights = [m for m in ["dsfb", "nis_soft"] if m in set(traj_df["method"].dropna().unique())]

if not weight_cols or not methods_for_weights:
    print("Weight columns or methods not available; skipping Figure 2.")
else:
    fig, axes = plt.subplots(len(methods_for_weights), 1, figsize=(8.0, 3.0 * len(methods_for_weights)), sharex=True)
    if len(methods_for_weights) == 1:
        axes = [axes]

    for ax, m in zip(axes, methods_for_weights):
        d = traj_df[traj_df["method"] == m].copy()
        g = d.groupby("t", as_index=False)[weight_cols].mean()
        for col in weight_cols:
            ax.plot(g["t"], g[col], linewidth=1.1, label=col)
        ax.set_title(f"{m} trust weights")
        ax.set_ylabel("weight")
        ax.legend(ncol=min(4, len(weight_cols)), fontsize=8)

    axes[-1].set_xlabel("t")
    fig.tight_layout()
    fig.savefig(FIG_DIR / "figure2_trust_weights.png")
    fig.savefig(FIG_DIR / "figure2_trust_weights.pdf")
    plt.show()


In [ ]:
# Figure 3: alpha/beta heatmap for peak_err (DSFB rows).
heat_dsfb = heatmap_df[heatmap_df["method"] == "dsfb"].copy()
if heat_dsfb.empty:
    print("No DSFB rows in heatmap.csv; skipping Figure 3.")
else:
    pivot = heat_dsfb.pivot_table(index="beta", columns="alpha", values="peak_err", aggfunc="mean")
    pivot = pivot.sort_index().sort_index(axis=1)

    fig, ax = plt.subplots(figsize=(6.0, 4.8))
    im = ax.imshow(pivot.values, aspect="auto", origin="lower", cmap="viridis")
    ax.set_title("DSFB Sweep: peak_err")
    ax.set_xlabel("alpha")
    ax.set_ylabel("beta")
    ax.set_xticks(range(len(pivot.columns)))
    ax.set_xticklabels([f"{v:.3g}" for v in pivot.columns], rotation=45, ha="right")
    ax.set_yticks(range(len(pivot.index)))
    ax.set_yticklabels([f"{v:.3g}" for v in pivot.index])
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label("peak_err")
    fig.tight_layout()
    fig.savefig(FIG_DIR / "figure3_peak_err_heatmap.png")
    fig.savefig(FIG_DIR / "figure3_peak_err_heatmap.pdf")
    plt.show()


In [ ]:
# Figure 4 (optional): alpha/beta heatmap for false_downweight_rate.
heat_dsfb = heatmap_df[heatmap_df["method"] == "dsfb"].copy()
heat_dsfb = heat_dsfb.dropna(subset=["false_downweight_rate"])

if heat_dsfb.empty:
    print("No numeric false_downweight_rate for DSFB; skipping Figure 4.")
else:
    pivot = heat_dsfb.pivot_table(index="beta", columns="alpha", values="false_downweight_rate", aggfunc="mean")
    pivot = pivot.sort_index().sort_index(axis=1)

    fig, ax = plt.subplots(figsize=(6.0, 4.8))
    im = ax.imshow(pivot.values, aspect="auto", origin="lower", cmap="magma")
    ax.set_title("DSFB Sweep: false_downweight_rate")
    ax.set_xlabel("alpha")
    ax.set_ylabel("beta")
    ax.set_xticks(range(len(pivot.columns)))
    ax.set_xticklabels([f"{v:.3g}" for v in pivot.columns], rotation=45, ha="right")
    ax.set_yticks(range(len(pivot.index)))
    ax.set_yticklabels([f"{v:.3g}" for v in pivot.index])
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label("false_downweight_rate")
    fig.tight_layout()
    fig.savefig(FIG_DIR / "figure4_false_downweight_heatmap.png")
    fig.savefig(FIG_DIR / "figure4_false_downweight_heatmap.pdf")
    plt.show()


## Outputs

Generated figures are saved to `./figures/` as both PNG and PDF:

- `figure1_error_trajectories.*`
- `figure2_trust_weights.*`
- `figure3_peak_err_heatmap.*`
- `figure4_false_downweight_heatmap.*` (if available)
